<a href="https://colab.research.google.com/github/RyosukeHanaoka/TechTeacher_New/blob/main/vit_eval_without_gradcam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install timm
import os
import torch
import timm
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 8.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manyl

In [3]:
class ImagePreprocessor:
    def __init__(self, size=(224, 224), mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]):
        self.size = size
        self.mean = mean
        self.std = std
        self.transform = transforms.Compose([
            transforms.Resize(self.size),
            transforms.ToTensor(),
            transforms.Normalize(mean=self.mean, std=self.std)
        ])

    def process_image(self, image_path, flip_left_hand=False):
        image = Image.open(image_path)
        if flip_left_hand:
            image = image.transpose(Image.FLIP_LEFT_RIGHT)
        image = image.convert('RGB')
        image = self.transform(image)
        return image

class RheumatoidArthritisModel:
    def __init__(self, checkpoint_path, model_name='vit_base_patch16_224_in21k'):
        self.model = timm.create_model(model_name, pretrained=False, num_classes=2)
        self.checkpoint_path = checkpoint_path
        self.load_checkpoint()
        self.model.eval()

    def load_checkpoint(self):
        checkpoint = torch.load(self.checkpoint_path, map_location=torch.device('cpu'))
        self.model.load_state_dict(checkpoint)

    def predict(self, image_tensor):
        with torch.no_grad():
            outputs = self.model(image_tensor.unsqueeze(0))
        return outputs

def main():
    input_directory_righthand = "/content/drive/MyDrive/image_righthand"
    output_directory_righthand = "/content/drive/MyDrive/converted_righthand"
    input_directory_lefthand = "/content/drive/MyDrive/image_lefthand"
    output_directory_lefthand = "/content/drive/MyDrive/converted_lefthand"

    preprocessor = ImagePreprocessor()

    right_hand_image_path = os.path.join(output_directory_righthand, "sample.jpg")
    left_hand_image_path = os.path.join(output_directory_lefthand, "sample.jpg")

    right_hand_image = preprocessor.process_image(right_hand_image_path)
    left_hand_image = preprocessor.process_image(left_hand_image_path, flip_left_hand=True)

    checkpoint_path = "/content/drive/MyDrive/OptPhotoFiles/model.pth"
    model = RheumatoidArthritisModel(checkpoint_path)

    right_hand_prediction = model.predict(right_hand_image)
    left_hand_prediction = model.predict(left_hand_image)

    print("Right Hand Prediction:", right_hand_prediction)
    print("Left Hand Prediction:", left_hand_prediction)

    if torch.argmax(right_hand_prediction) == 1 or torch.argmax(left_hand_prediction) == 1:
        print("Rheumatoid Arthritis Detected")

if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name vit_base_patch16_224_in21k to current vit_base_patch16_224.augreg_in21k.
  model = create_fn(


Right Hand Prediction: tensor([[1.7585, 2.5228]])
Left Hand Prediction: tensor([[2.3716, 2.0022]])
Rheumatoid Arthritis Detected
